In [ ]:
# Agent ID 설정
agent_id = "YOUR_AGENT_ID"  # Agent ID
agent_alias_id = "TSTALIASID"  # Test Agent Alias ID that is created initialization step
session_id = "test-session-1"
region_name = "us-west-2"

In [2]:
import boto3
from botocore.config import Config
import json
import time

# Bedrock 클라이언트 생성
bedrock_agent_runtime = boto3.client(
    service_name='bedrock-agent-runtime',
    region_name=region_name,
    config=Config(read_timeout=3600)
)

In [3]:
def ask_agent(question):
    print(f"\nQ: {question}")
    
    try:
        response = bedrock_agent_runtime.invoke_agent(
            agentId=agent_id,
            agentAliasId=agent_alias_id,
            sessionId=session_id,
            inputText=question
        )
        
        # EventStream 처리
        completion = response['completion']
        
        print("A: ", end='', flush=True)
        # EventStream의 각 이벤트를 처리하며 실시간 출력
        for event in completion:
            if 'chunk' in event:
                chunk_data = event['chunk']['bytes'].decode()
                try:
                    # JSON 형식으로 파싱 시도
                    chunk_json = json.loads(chunk_data)
                    if isinstance(chunk_json, dict) and 'completion' in chunk_json:
                        text = chunk_json['completion']
                    else:
                        text = str(chunk_json)
                except json.JSONDecodeError:
                    text = chunk_data
                
                # 한 글자씩 출력
                for char in text:
                    print(char, end='', flush=True)
                    time.sleep(0.02)  # 출력 속도 조절
        
        print()  # 줄바꿈
    
    except Exception as e:
        print(f"Error: {str(e)}")

In [4]:
# example questions
questions = [
    "AWS와 관련된 뉴스 3개를 알려주세요.",
#     "Tell me three AWS related news stories.",
    "현재 서울의 시간을 HH:mm:ss 형식으로 알려주세요.",
#     "Tell me the current time in Seoul using the format HH:mm:ss."
]

# ask multiple questions
for question in questions:
    answer = ask_agent(question)


Q: AWS와 관련된 뉴스 3개를 알려주세요.
A: 다음은 AWS와 관련된 최신 뉴스 3개입니다:

1. **What's New at AWS - Cloud Innovation & News**
   - URL: [https://aws.amazon.com/new/](https://aws.amazon.com/new/)
   - 요약: AWS는 지속적으로 새로운 기능을 추가하여 최신 기술을 활용하여 더 빠르게 실험하고 혁신할 수 있도록 합니다. What's New 게시물은 우리가 바로 그렇게 하고 있음을 보여주며, 모든 AWS 서비스, 기능 및 지역 확장 발표가 출시될 때마다 간략한 개요를 제공합니다.

2. **Latest news about Amazon and AWS**
   - URL: [https://www.aboutamazon.com/amazon-aws-news](https://www.aboutamazon.com/amazon-aws-news)
   - 요약: Amazon Web Services(AWS)에 대한 최신 뉴스 및 업데이트를 읽으세요. Bedrock, Q 및 기타 도구의 최신 개발 사항, 기술 교육, 커뮤니티 이니셔티브 등에 대해 알아보세요. ... Amazon Web Services(AWS)는 200개 이상의 완전한 기능을 갖춘 서비스를 데이터에서 사용할 수 있는 세계에서 가장 포괄적이고 널리 채택된 클라우드입니다.

3. **The 10 Biggest AWS News Stories Of 2024: CEO Exit, AI And Partners - CRN**
   - URL: [https://www.crn.com/news/cloud/2024/top-10-biggest-aws-news-stories-of-2024-ceo-exit-ai-and-partners](https://www.crn.com/news/cloud/2024/top-10-biggest-aws-news-stories-of-2024-ceo-exit-ai-and-partners)
   - 